In [ ]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
tf.logging.set_verbosity(tf.logging.ERROR)
mnist = input_data.read_data_sets("/Users/kensuketamura/GoogleDrive_tamuken.phys@gmail.com/program/data/mnist_data/", one_hot = True) #指定ディレクトリにファイルがない場合はダウンロード

images= mnist.train.images #訓練用の画像
labels = mnist.train.labels #訓練画像のラベル
test_images = mnist.test.images #テスト用の画像
test_labels = mnist.test.labels #テスト画像のラベル

learning_rate = 0.1
training_epochs = 2000
batch_size = 50
dropout = 0.5

with tf.name_scope("input"):
    x = tf.placeholder(tf.float32, [None, 784], name="input") 
    out_img = tf.reshape(x, [-1, 28, 28, 1]) #xというプレースホルダーを(-1, 28, 28, 1)の形に変形（これにより入力のニューロン数は728から1になったとみなされる。）
    
#畳み込み層(28, 28, 1) -> (28, 28, 32)
with tf.name_scope("layer1_conv"):
    f1 = tf.Variable(tf.truncated_normal([3, 3, 1, 32], # [フィルターの縦, フィルターの横, 画像のチャンネル数, フィルターの枚数]
                     stddev = 0.1)) #平均0 標準偏差0.1 の分布の標準偏差|2|の範囲で値を取得
    conv1 = tf.nn.conv2d(out_img, f1, 
                         strides=[1, #入力データは１ずつconv 
                                  1, #フィルターは縦方向に1ずつずらす
                                  1, #フィルターは横方向に１ずつずらす
                                  1],#チャンネルのストライド
                         padding="SAME") ##フィルターの大きさだけを考慮してpadding(従って出力の画像サイズは単に（元のサイズ/ストライド）で計算できる, 多分。)
    
    b1 = tf.Variable(tf.constant(0.1, #0.1で初期化
                                 shape=[32])) #フィルターの枚数分用意しておく
    
    out_conv1 = tf.nn.relu(conv1 + b1) #ReLUでactivate

#pooling層   (28, 28, 32) -> (14, 14, 32)
with tf.name_scope("layer2_pooling"):
    out_pool1 = tf.nn.max_pool(out_conv1, 
                               ksize=[1, #poolingを行うwindowの大きさ: 入力するデータに対する幅(out_conv1の0次元目)
                                      2, #データの縦方向に対する幅(out_conv1の1次元目)
                                      2, #データの横方向に対する幅(out_conv1の2次元目)
                                      1],#データのチャンネル方向に対する幅(out_conv1の３次元目)
                               strides=[1, 2, 2, 1], #
                               padding="SAME") #フィルターの大きさだけを考慮してpadding(従って出力の画像サイズは単に（元のサイズ/ストライド）で計算できる)
    
#畳み込み層 (14, 14, 32) -> (14, 14, 64)
with tf.name_scope("layer3_conv"):
    f2 = tf.Variable(tf.truncated_normal([3, #フィルターの縦方向のサイズ
                                          3, #フィルターの横方向のサイズ
                                          32, #フィルターのチャンネル方向のサイズ
                                          64], #フィルターの枚数
                                         stddev=0.1))
    
    conv2 = tf.nn.conv2d(out_pool1, 
                         f2, 
                         strides=[1, 1, 1, 1],
                         padding = "SAME")
    b2 = tf.Variable(tf.constant(0.1, shape=[64]))
    out_conv2 = tf.nn.relu(conv2 + b2)
    
#pooling層(14, 14, 64) -> (7, 7, 64)
with tf.name_scope("layer4_pooling"):
    out_pool2 = tf.nn.max_pool(out_conv2, 
                               ksize=[1, 2, 2, 1], 
                               strides=[1, 2, 2, 1], 
                               padding="SAME")
    
#dropout
with tf.name_scope("dropout"):
    keep_prob = tf.placeholder(tf.float32) #非ドロップアウトのニューロン数の割合。学習時は0.5など。テスト時には1.0に変えられるようにplaceholderとして用意。
    out_drop = tf.nn.dropout(out_pool2, keep_prob)
    
with tf.name_scope("flatten"):
    out_flat = tf.reshape(out_drop, [-1, 7*7*64]) #データ数×(7*7*64の1次元データ)に変換
    
#全結合層(7*7*64) -> (1024)
with tf.name_scope("layer5_binding"):
    w1 = tf.Variable(tf.truncated_normal([7*7*64, 1024], stddev=0.1))
    b1 = tf.Variable(tf.constant(0.1, shape=[1024]))
    out1 = tf.nn.relu(tf.matmul(out_flat, w1) + b1)

#出力層(1024) -> (10)    
with tf.name_scope("layer6_output"):
    w2 = tf.Variable(tf.truncated_normal([1024, 10], stddev=0.1))
    b2 = tf.Variable(tf.constant(0.1, shape=[10]))
    out = tf.nn.softmax(tf.matmul(out1, w2) + b2)
    
#損失関数(cross entropy)
with tf.name_scope("loss"):
    t = tf.placeholder(tf.float32, shape=[None, 10], name="label" )
    loss = tf.reduce_mean(-tf.reduce_sum(t*tf.log(out + 1e-5), axis=[1]))

#訓練
with tf.name_scope("train"):
    train_op = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)
with tf.name_scope("accuracy"):
    correct = tf.equal(tf.argmax(out, 1), tf.argmax(t, 1)) #予測値と答えが一致しているかチェック
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
    
init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    
    for step in range(training_epochs):
        train_images, train_labels = mnist.train.next_batch(batch_size)
        sess.run(train_op, 
                 feed_dict={x:train_images, t:train_labels, keep_prob:dropout})
        
        step += 1
        if step % 100 == 0:
            acc_val = sess.run(accuracy, feed_dict={x:test_images, t:test_labels, keep_prob:1.0})
            
            print("Step{}: acccuracy = {}".format(step, acc_val))

In [ ]:
train_im = images.reshape([-1, 28, 28, 1])
img_id = 0 #フィルターを適用する画像のインデックス
vertical_edge_fil = np.array([[-2, 1, 1],
                              [-2, 1, 1],
                              [-2, 1, 1]], dtype=float) #フィルター(縦方向を検出)
horizontal_edge_fil = np.array([[1, 1, 1],
                                [1, 1, 1],
                                [-2, -2, -2]], dtype=float)
img_x = train_im[img_id, :, :, 0] #img_id番目の画像のピクセル値を取得
size = img_x.shape #(28, 28)
img_height = size[0]
img_width = size[1]
vertical_edge = np.zeros_like(img_x) #img_xと同じ形のzerosを生成
horizontal_edge = np.zeros_like(img_x) 

#フィルター適用
for h in range(img_height - 3):
    for w in range(img_width - 3):
        img_region = img_x[h:h+3, w:w+3] #フィルターの適用領域
        vertical_edge[h + 1, w + 1] = np.dot(img_region.reshape(-1), vertical_edge_fil.reshape(-1)) #それぞれ1次元データにしてから内積を取る
        horizontal_edge[h + 1, w + 1] = np.dot(img_region.reshape(-1), horizontal_edge_fil.reshape(-1))
plt.figure(figsize=(12, 3))
plt.subplots_adjust(wspace=0.5)
plt.gray()
plt.subplot(1, 3, 1) #(1 × 3のグリッドの左にプロット)
plt.pcolor(1 - img_x) #色を反転
plt.xlim(-1, 29)
plt.ylim(29, -1)
plt.subplot(1, 3, 2)
plt.pcolor(-vertical_edge)
plt.xlim(-1, 29)
plt.ylim(29, -1)
plt.subplot(1, 3, 3)
plt.pcolor(-horizontal_edge)
plt.xlim(-1, 29)
plt.ylim(29, -1)

In [31]:
7*7*64

3136